In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

# Limitaciones de DQN

Las Deep Q Networks (y Q-Learning) tienen dos limitaciones importantes
- Espacio de acción discreto (y espacio de estados discreto)
- Requieren heurísticas para que haya exploración 

Estos métodos se basaban en estimar la función **Q** 

Luego se usa una política determinista de máximas utilidades

$$
\pi(s) = \text{arg}\max_a' Q(s, a')
$$

Una alternativa es aprender directamente la *policy*. Veremos como esto puede resolver las limitaciones que mencionamos al principio

# Policy learning

En lugar de aprender **Q** podemos aprender directamente

$$
a \sim \pi_\theta (s)
$$

- La política es estocástica, es decir sigue una distribución de probabilidad
- La acción se muestrea de la política: Esto nos da exploración

Podemos usar una red neuronal para modelar los parámetros de la distribución

#### Ejemplo

Espacio de acción discreto con 3 opciones

Usamos una política con distribución categórica

Usamos una red neuronal con tres salidas correspondientes a $p(a_1|s)$, $p(a_2|s)$  y $p(a_3|s)$, donde cada una está en el rango $[0,1]$  y además suman uno: **salida softmax**

Luego muestreamos de la distribución categórica usando los parámetros entregados por la red

In [ ]:
from scipy.stats import multinomial

neural_net_output = np.array([1/3, 1/3, 1/3])
#neural_net_output = np.array([0.7, 0.1, 0.2])

samples = multinomial(n=1, p=neural_net_output).rvs(10000)

fig, ax = plt.subplots(figsize=(7, 3))
ax.hist(np.argmax(samples, axis=1));

#### Ejemplo

Espacio de acción con dos acciones continuas

Usamos una política con distribución Gaussiana

Usamos una red neuronal con cuatro salidas (dos por acción) correspondientes a $\mu(a_1|s)$, $\log \sigma(a_1|s)$, $\mu(a_2|s)$ y $\log \sigma(a_2|s)$, donde cada una está en el rango de los reales: **salida lineal**

Luego muestreamos de la distribución Gaussiana usando los parámetros entregados por la red

In [ ]:
from scipy.stats import multivariate_normal

neural_net_output = np.array([0.5, 2., np.log(0.5), np.log(3.)])

samples = multivariate_normal(mean=neural_net_output[:2], 
                              cov=np.diag(np.exp(neural_net_output[2:]))).rvs(10000)

fig, ax = plt.subplots(figsize=(7, 3))
ax.hist(samples[:, 0], bins=100, alpha=0.5, density=True);
ax.hist(samples[:, 1], bins=100, alpha=0.5, density=True);

# Algoritmo de entrenamiento

El algoritmo que usaremos para entrenar busca lo siguiente:
- Correr política hasta el término del episodio, grabando las tuplas de acción/estado/recompensa
- Disminuir la probabilidad de las acciones que terminaron en recompensa baja
- Aumentar la probabilidad de las acciones que terminaron en recompensa alta

Esto se peude resumir con la siguiente loss:

$$
\mathcal{L}(s_t, a_t, r_t, \mathcal{H}) = - \log P(a_t|s_t) G_t
$$

donde
- Log verosimilitud $\log P(a_t|s_t)$: mide que tan posible es haber seleccionado $a_t$
- Ganancia total descontada $G_t$: La ganacia recibida por seleccionar $a_t$

Si entrenamos usando gradiente descedente entonces

$$
\theta \leftarrow \theta + \eta \nabla \log P(a_t|s_t) G_t
$$

De donde sale el nombre de este algoritmo: *policy gradient*


In [ ]:
import torch

# Modelo para los parámetros de la política
class MultilayerPerceptron(torch.nn.Module):    
    def __init__(self, n_input, n_output, n_hidden=10):
        super(type(self), self).__init__()
        self.linear1 = torch.nn.Linear(n_input, n_hidden)
        self.linear2 = torch.nn.Linear(n_hidden, n_hidden)
        self.linear3 = torch.nn.Linear(n_hidden, n_output)
        self.activation = torch.nn.ReLU()
        
    def forward(self, x):
        h = self.activation(self.linear1(x))
        h = self.activation(self.linear2(h))
        return  self.linear3(h)
    
# Memoría para guardar lo que ocurre en un episodio
class Memory:
    def __init__(self): 
        self.reset()
    
    def reset(self): 
        self.s = []
        self.a = []
        self.r = []

    def push(self, s, a, r): 
        self.s.append(s)
        self.a.append(a)
        self.r.append(r)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.signal import convolve

fig, ax = plt.subplots(2, figsize=(6, 3), sharex=True, tight_layout=True)

def smooth_data(x, window_length=10):
    return convolve(x, np.ones(window_length)/window_length, mode='valid')

def update_plot(step, episode, smooth_window=10, target=195, target_update=500):
    for ax_ in ax:
        ax_.cla()
    episodes = np.arange((episode))
    ax[0].scatter(episodes[:episode], diagnostics['rewards'][:episode], s=1)      
    if episode > smooth_window:
        ax[0].plot(episodes[:-smooth_window+1], 
                   smooth_data(diagnostics['rewards'][:episode]), alpha=0.5, lw=2)        
        #ax[1].plot(episodes[:-smooth_window+1], 
        #           smooth_data(diagnostics['loss'][:episode]))
    ax[1].plot(episodes, diagnostics['loss'][:episode])
    ax[0].plot(episodes, [target]*len(episodes), 'k--')
    ax[0].set_ylabel('Recompensa');
    ax[1].set_ylabel('Loss')
    ax[1].set_xlabel('Episodios')
    ax[0].set_title("Paso %d" % (step))
    fig.canvas.draw()

In [ ]:
import numpy as np
import gym
from tqdm.notebook import tqdm

torch.manual_seed(123)

env = gym.make("CartPole-v0")
n_state = env.observation_space.shape[0] # Número de estados
n_action = env.action_space.n # Número de acciones

model = MultilayerPerceptron(n_state, n_action)

gamma = 0.999
loglikelihood = torch.nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

memory = Memory()

num_episodes = 3000
diagnostics = {'rewards': np.zeros(shape=(num_episodes,)), 
               'loss': np.zeros(shape=(num_episodes,)),
               'target_updates': []}

global_step_counter = 0
for episode in tqdm(range(num_episodes)):
    state = env.reset()
    end = False
    
    while not end:        
        # Escoger acción
        state = torch.tensor(state).float()  
        with torch.no_grad():
            logits = model.forward(state)
            probs = torch.nn.Softmax(dim=0)(logits).numpy()
        a = np.random.choice(n_action, size=1, p=probs)[0]
        # Ejecutar acción
        s, r, end, info = env.step(a)
        # Guardar en memoria
        memory.push(state, torch.tensor(a, dtype=int), torch.tensor(r))
        # Preparar para próximo paso
        diagnostics['rewards'][episode] += r
        state = s
        global_step_counter +=1
        
    # Una vez completado el episodio actualizamos el modelo
    
    # Primero calculamos la recompensa total descontada
    Gt = (torch.stack(memory.r)*gamma**torch.arange(0, len(memory.r)).float()).flip(0).cumsum(0).flip(0)
    # Luego la normalizamos (mejora la estabilidad del entrenamiento)
    Gt = (Gt - Gt.mean())/Gt.std()
    # Predicción (sin normalizar) de la probabilidad de acción
    logits = model.forward(torch.stack(memory.s))
    # Cálculo de la loss
    loss = torch.mean(loglikelihood(logits, torch.stack(memory.a))*Gt)
    # Calculamos los gradientes y actualizamos los parámetros
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()     
    diagnostics['loss'][episode] += loss
    # Borramos la memoria
    memory.reset()
                
    if episode % 25 == 0:
        update_plot(global_step_counter, episode)

Agente en acción

In [ ]:
import numpy as np
import gym

env = gym.make("CartPole-v0")

In [ ]:
env.reset()
end = False

for k in range(500):
    env.render()
    state = torch.from_numpy(np.array(env.state).astype('float32'))
    with torch.no_grad():
        logits = model.forward(state)
        probs = torch.nn.Softmax(dim=0)(logits).numpy()
    a = np.random.choice(2, size=1, p=probs)[0]
    s_future, r, end, info = env.step(a)
    #if end:
    #    break
    if r == 0:
        display(k)
        break
display(end, k)

In [ ]:
env.close()

# AlphaGo 

Go es un juego de mesa de origen chino con un tablero de 19x19 espacios

Dos jugadores compiten por cubrir la mayor área en el tablero

El espacio de estados es tiene [más movimientos legales que átomos en el universe](https://en.wikipedia.org/wiki/Go_and_mathematics#Legal_positions)

El equipo de DeepMind ha utilizado Go como benchmark para proponer nuevos modelos de RL profundo

El primero de ellos es Alpha Go (2016) el cual combinó bastantes técnicas
1. Se entrena una red neuronal supervisada con movimientos de humanos expertos: Imitar a los humanos
1. Se entrena un agente basado en política que compite contra el modelo entrenado: Aprender nuevos movimientos
1. Una vez que el agente supera al modelo supervisado se sigue entrenando contra si mismo
1. Adicionalmente se ocupa un modelo basado en valor para evaluar los movimientos del agente

[Presentación autocontenida sobre Alpha Go](https://www.slideshare.net/ckmarkohchang/alphago-in-depth)

Alpha Go Zero no usa pre-entrenamiento con redes supervisadas

Alpha Zero extiende a otros juegos

## Desafíos de Aprendizaje Reforzado

- Para entrenar con *policy gradient* necesitamos correr el agente hasta el término del episodio
- Esto en muchas cosas significa la muerte del agente
- ¿Cómo podemos desarrollar un agente en el mundo real?

Una alternativa es usar simuladores
- Usar un simulación realista del ambiente para entrenar, e.g. [VISTA](http://www.mit.edu/~amini/vista/)
- Hacer transferencia de aprendizaje desde el simulador al mundo real (por las imperfecciones del simulador)

Otra alternativa es one-shot learning
- [One-shot learning of manipulation skills with online dynamics adaptation and neural network priors](http://rll.berkeley.edu/iros2016onlinecontrol/online_control.pdf)

Y otra alternativa más es usar apoyo humano
- [Trial without Error: Towards Safe Reinforcement Learning via Human Intervention](https://arxiv.org/abs/1707.05173) y [blog post](https://owainevans.github.io/blog/hirl_blog.html)


Otro desafío general de RL es el proponer buenas funciones de recompensa. Este y otros desafios abiertos se discuten en esta [
fuerte crítica a RL](https://www.alexirpan.com/2018/02/14/rl-hard.html)